# ***RULER (a RULe based approach for Eliciting Requirements)***

# Installing dependencies

In [1]:
#Installing dependencies

!pip install owlready2
import spacy
from owlready2 import *
from google.colab import files
import re
import inflect

nlp = spacy.load("en_core_web_sm")
#download the ontology file at: https://drive.google.com/file/d/1IC5ffOtcuLQE-8RYdWbf7P4Eqw_JeVJU/view?usp=sharing and upload it to your workspace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Processing meta-ontology for RE and answers

In [2]:

def get_question_focus(question):

  inf = inflect.engine()
  get_singular = lambda x: inf.singular_noun(x)
  get_plural = lambda x: inf.plural(x)

  #Pattern
  focus_matches = re.search('^what (are|is) the ([a-zA-Z].*) (of|in)\s?(the)?\s?(\"[-\[\]\*\?!#$%&a-zA-Z0-9].*\")', question)
  focus_feature = focus_matches.group(2)
  focus_concept = focus_matches.group(5)
  focus_concept = focus_concept.replace('"', '')

  feature_singular = get_singular(focus_feature) 
  focus_feature =  focus_feature if feature_singular == False else feature_singular #focus head
  return focus_concept, focus_feature 
  
#Classes-attributes(object and data properties)
def get_ontology_structure(ontology):

  #Inherited attributes
  def add_inherited_attributes():
    for ont_class in ontology_classes:
      class_superclasses = ont_class.is_a
      if(class_superclasses[0].name != "Thing"):
        for superclass in class_superclasses:
          ontology_structure[ont_class.name] += ontology_structure[superclass.name]
    return ontology_structure

  #Removing superclasses
  def remove_superclasses():
    for superclass in superclasses:
      del(ontology_structure[superclass.name])
    return ontology_structure
  
  ontology_classes = [ont_class for ont_class in ontology.classes()]
  ontology_properties = [ont_property for ont_property in ontology.properties()]
  ontology_structure = dict((ont_class.name, []) for ont_class in ontology_classes) #class-attributes
  superclasses = [ont_class for ont_class in ontology.classes() if len(list(ont_class.subclasses())) > 0]

  get_property_range_name = lambda x: "none" if type(x) is type else x.name

  for ont_property in ontology_properties:
    ont_property_features = eval(ont_property.comment[0])
    ont_property_features["range"] = get_property_range_name(ont_property.range[0])
    ontology_structure[ont_property.domain[0].name].append((ont_property.name, ont_property_features))

  #Additional steps
  ontology_structure = add_inherited_attributes()
  #ontology_structure = remove_superclasses()
  
  return ontology_structure

In [3]:
def building_ont_dict(answer_dict, f_individual, property_range, property_name,individual):
  #---- building the dict for populating the ontology
  if(property_range == "Actor"):
    if(answer_dict["discourse"]["concept"].get(f_individual) is None):
      answer_dict["discourse"]["concept"][f_individual] = {"type":"actor", "attribute":[]}
  elif(property_range == "Concept"):
    existent_concept = answer_dict["discourse"]["concept"].get(f_individual)
    if(existent_concept is None): #concept-object type, not actor
      answer_dict["discourse"]["concept"][f_individual] = {"type":"object", "attribute":[]}
  elif(property_range == "Action"): 
    action_parts = f_individual.split("*")
    f_action_verb = action_parts[0]
    f_related_concept = action_parts[1]
    answer_dict["discourse"]["action"][f"{individual}**{f_action_verb}**{f_related_concept}"] = {"f_related_actor":individual,
                                                                                                   "f_action_verb":f_action_verb,
                                                                                                   "f_related_concept":f_related_concept}
  if(property_name == "attributes"):
    answer_dict["discourse"]["concept"][individual]["attribute"].append(f_individual)
  return answer_dict

def generate_question(ontology, answer_dict={"discourse":{"concept":{},"action":{}}}, answer_dictionary=None, individual="discourse", query_class="Discourse", validation_dictionary={}):
  import re
  import inflect
  from termcolor import colored, cprint

  inflect = inflect.engine()
  ontology_structure = get_ontology_structure(ontology)
  if(answer_dictionary is None):
    answer_dictionary = dict((key, []) for key in ontology_structure.keys())
  
  beautify_property_str = lambda x: " ".join(re.findall("[A-Z][^A-Z]*", x[3:])).lower()
  beautify_individual_str = lambda x: " ".join(individual.split("_"))

  #First question ---> Discourse 'has-a' relationships (default)
  wh_auxiliaries = {"single":"is", "multiple":"are"}
  ontology_classes = list(ontology_structure.keys())

  for ont_property in ontology_structure[query_class]:
    property_name, property_features = ont_property
    property_name = beautify_property_str(property_name)
    is_property_askable, property_cardinality, property_range = map(lambda x: property_features[x], 
                                                                              property_features.keys())
    property_name = inflect.plural(property_name) if property_cardinality == "multiple" else property_name

    question = "what %s the %s %s \"%s\"? " % (wh_auxiliaries[property_cardinality], 
                                               property_name,
                                               "in" if property_name == "related concepts" else "of the", 
                                               individual)
    
    printing_question = question.replace("*", " ").strip()

    if(is_property_askable):
      #printing only askable questions
      print("*",colored(printing_question, attrs=['bold', 'dark']), colored("---> askable", "green") if is_property_askable else colored("---> not askable", "red"))
      
      #related_concept case --> * func * obj
      patt_match = question.find("*")
      if(patt_match != -1):
        question_copy = question[patt_match+1:]
        right_end = question_copy.find("*")
        focus_concept = question_copy[:right_end].strip()[:-2]
        focus_feature = "related concept"
      else:
        focus_concept, focus_feature = get_question_focus(question)

      #controlling related concepts
      if(focus_feature == "related concept"):
        answer = focus_concept
      else:
        question_data = {"text":question,
                         "scope":"general" if focus_concept == "discourse" else "specific",
                         "focus_concept":focus_concept,
                         "focus_feature":focus_feature}
        answer = input() 
      validation_dictionary[printing_question] = answer 
      
      if(len(answer)>0):
        answer = list(map(lambda x: x.strip(),answer.split(",")))
        print(colored("Answer ---> ", "yellow", attrs=["bold"]), answer)

        #Avoiding deep asking for additional roles.
        if(focus_feature != "additional role"):
          for f_individual in answer:
            answer_dict = building_ont_dict(answer_dict, f_individual, property_range, property_name,individual)
            #----
            if(f_individual not in answer_dictionary[property_range]):
              answer_dictionary[property_range].append(f_individual)
              #-> preventing question repetition 
              #questions based on the property type
              if(property_name in ontology_classes):
                generate_question(ontology, answer_dict=answer_dict, answer_dictionary=answer_dictionary, individual=f_individual, query_class=property_name)
              #questions based on the range type
              elif(property_range not in ["none", ""]):
                generate_question(ontology, answer_dict=answer_dict, answer_dictionary=answer_dictionary, individual=f_individual, query_class=property_range)
      else:
        print(colored("Answer ---> ", "yellow", attrs=["bold"]), "None")
    else: print(colored(question, "red", attrs=["bold"]))
  return answer_dictionary, answer_dict, validation_dictionary

# Populating the meta-ontology for RE

In [4]:
#Populating OntRE WORKFLOW
'''
Actor --> Actor in discourse
Function 
--> Object --> hasName(object) 
--> Action --> hasActionVerb(verb)
--> Action --> hasRleatedActor (actor loop)
--> Action --> hasRelatedObject (object forecreated)
Attribute 
--> Object --> hasName (object)
--> Concept --> hasAttribute (object)
'''

#POPULATING THE ONTOLOGY!!!! 
'''
1. for each concept --> create actor or object according to concept_type
2. for each concept --> fill hasAttribute property                        
3. for each action --> create action and fill properties (related_actor, related_object)
'''

'\n1. for each concept --> create actor or object according to concept_type\n2. for each concept --> fill hasAttribute property                        \n3. for each action --> create action and fill properties (related_actor, related_object)\n'

In [5]:
def create_object_individual(ontology, object_name):
  object_class = ontology.Object
  object_individual = object_class(object_name, hasName=object_name)
  return object_individual

#actor_attributes = [(object_name, object_attributes)...]
def create_actor_individual(ontology, actor_role):
  actor_class = ontology.Actor
  actor_individual = actor_class(actor_role, hasRole = actor_role)
  ontology.discourse.hasDiscourseActor.append(actor_individual)
  return actor_individual

def add_concept_attribute(concept_individual, concept_attribute):
  concept_individual.concept_attribute.append(concept_attribute)
  return concept_individual

def create_action_individual(ontology, name, hasRelatedActor_str, hasRelatedConcept_str, hasActionVerb):
  action_class = ontology.Action
  hasRelatedActor = ontology.search_one(iri="*"+hasRelatedActor_str, is_a=ontology.Actor)
  hasRelatedConcept = ontology.search_one(iri="*"+hasRelatedConcept_str, is_a=ontology.Concept)

  action_individual = action_class(name, hasActionVerb=hasActionVerb)
  if(hasRelatedActor is not None):
    action_individual.hasRelatedActor.append(hasRelatedActor)
  else:
    print("NONE")
  if(hasRelatedConcept is not None):
    action_individual.hasRelatedConcept.append(hasRelatedConcept)
  else:
    print("NONE")

  return action_individual

def destroy_entities(ontology): 
  actors = ontology.search(is_a=ontology.Actor)
  objects = ontology.search(is_a=ontology.Object)
  actions = ontology.search(is_a=ontology.Action)
  #print(actors, objects, actions)
  a = [actors, objects, actions]
  for i in a:
    for j in i[1:]:
      destroy_entity(j)

  return ontology

#Populating the ontology for RE
def populate_ontology(ontology, ontology_data, ontology_save_path):
  #clean_ontology
  ontology = destroy_entities(ontology)

  concept_keys = ontology_data["discourse"]["concept"].keys()
  #print(f"concept_keys {concept_keys}")
  action_keys = ontology_data["discourse"]["action"].keys()
  pp_name = lambda x: "_".join(x.split(" "))
  ontre_dict = {"actor":ontology.Actor, "object":ontology.Object}
  create_concept_switch = {"actor": lambda x,y: create_actor_individual(x,y), "object": lambda x,y: create_object_individual(x,y)}
  nlp_concepts = list(nlp.pipe(concept_keys, disable=["ner"]))
  #Adding concepts (Actor and Object type)
  for idx,concept in enumerate(concept_keys):
    
    concept_data = ontology_data["discourse"]["concept"][concept]
    concept_type = concept_data["type"]
    concept_attributes = concept_data["attribute"]
    concept = "_".join([tk.lemma_ for tk in nlp_concepts[idx]]).lower()
    #print(f"concept {concept}")
    #concept = pp_name(concept)
    
    if(concept_type == "actor"):
      concept_individual = create_actor_individual(ontology, concept)
      print(f"actor {concept}")
    elif(concept_type == "object"):
      print(f"concept {concept}")
      concept_individual = create_object_individual(ontology, concept)

    nlp_attributes = list(nlp.pipe(concept_attributes, disable = ["ner"]))
    nlp_attributes = ["_".join([tk.lemma_ for tk in doc]).lower() for doc in nlp_attributes]
    #print(f"attributes {concept_attributes}")
    for idx,attribute in enumerate(concept_attributes):
      attribute_individual = create_concept_switch[ontology_data["discourse"]["concept"][attribute]["type"]](ontology,nlp_attributes[idx])
      concept_individual.hasAttribute.append(attribute_individual) 
      #print(attribute_individual, concept_individual) 

  #Adding actions (hasRelatedActor, hasActionverb, hasRelatedConcept)
  for action in action_keys:
    #print(f"action {action}")
    action_data = ontology_data["discourse"]["action"][action]
    f_action_verb = action_data["f_action_verb"]
    f_related_concept = action_data["f_related_concept"]
    f_related_actor = action_data["f_related_actor"]

    #beautifying names
    #f_action_verb, f_related_concept, f_related_actor = map(pp_name,[f_action_verb, f_related_concept, f_related_actor])
    f_action_verb,f_related_concept,f_related_actor = map(lambda x: "_".join([tk.lemma_ for tk in nlp(x)]).lower(), [f_action_verb, f_related_concept, f_related_actor])
    action_name = f"{f_related_actor}*{f_action_verb}*{f_related_concept}"
    print(f"action_name {action_name}, f_related_actor {f_related_actor}, f_related_concept {f_related_concept}, f_action_verb {f_action_verb}")
    action_individual = create_action_individual(ontology, action_name, f_related_actor, f_related_concept, f_action_verb)

    #adding hasaction to relatedActor
    actor_individual = ontology.search_one(iri="*"+pp_name(f_related_actor), is_a=ontology.Actor)
    if(actor_individual is not None):
      actor_individual.hasAction.append(action_individual)
    else:
      print("NONE")
  ontology.save(file=ontology_save_path, format="rdfxml")

# Main! (Start here)

**Instructions**

- Download the meta-ontology for RE example: https://drive.google.com/file/d/1IC5ffOtcuLQE-8RYdWbf7P4Eqw_JeVJU/view?usp=sharing
- Upload it to your workspace --> update the names in line 3 on the next cell if you change it
- Go to the menu->Runtime->Run all or (Ctrl+F9)
- Ask and answer the questions
- Download the owl file with the populated meta-ontology for RE
- You may visualize it by using protégé: https://protege.stanford.edu/ (it's open source)

In [6]:
#Main
ontre = get_ontology("file://ontre_ex.owl").load() 
RULER_data = generate_question(ontre)
populate_ontology(ontre,RULER_data[1],"domain_rep.owl")
files.download("domain_rep.owl")

what are the discourse actions of the "discourse"? 
* what are the discourse actors of the "discourse"? ---> askable
user,admin
Answer --->  ['user', 'admin']
what are the roles of the "user"? 
* what are the actions of the "user"? ---> askable
uploads*picture
Answer --->  ['uploads*picture']
what are the action verbs of the "uploads*picture"? 
what are the related actors of the "uploads*picture"? 
* what is the related concept of the "uploads picture"? ---> askable
Answer --->  ['picture']
* what are the attributes of the "picture"? ---> askable
size
Answer --->  ['size']
* what are the attributes of the "size"? ---> askable

Answer --->  None
* what are the attributes of the "user"? ---> askable
name
Answer --->  ['name']
* what are the attributes of the "name"? ---> askable

Answer --->  None
what are the roles of the "admin"? 
* what are the actions of the "admin"? ---> askable
adds*user
Answer --->  ['adds*user']
what are the action verbs of the "adds*user"? 
what are the related 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>